In [1]:
import geopandas as gpd
import pandas as pd
import pickle
import momepy
import dask
from dask.distributed import Client, LocalCluster
from tqdm.auto import tqdm
from scripts.percentile import Percentiles


/tmp/ipykernel_13379/1586022436.py:1: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/latest/migration_pygeos.html).
  import geopandas as gpd


In [2]:
daskCluster = LocalCluster(threads_per_worker=2,
                n_workers=8, memory_limit='100GB')

client = Client(daskCluster, timeout="300s")

client



2023-04-25 17:12:56,823 - distributed.nanny.memory - WARNING - Ignoring provided memory limit 100GB due to system memory limit of 31.31 GiB
2023-04-25 17:12:56,826 - distributed.nanny.memory - WARNING - Ignoring provided memory limit 100GB due to system memory limit of 31.31 GiB
2023-04-25 17:12:56,828 - distributed.nanny.memory - WARNING - Ignoring provided memory limit 100GB due to system memory limit of 31.31 GiB
2023-04-25 17:12:56,830 - distributed.nanny.memory - WARNING - Ignoring provided memory limit 100GB due to system memory limit of 31.31 GiB
2023-04-25 17:12:56,832 - distributed.nanny.memory - WARNING - Ignoring provided memory limit 100GB due to system memory limit of 31.31 GiB
2023-04-25 17:12:56,835 - distributed.nanny.memory - WARNING - Ignoring provided memory limit 100GB due to system memory limit of 31.31 GiB
2023-04-25 17:12:56,837 - distributed.nanny.memory - WARNING - Ignoring provided memory limit 100GB due to system memory limit of 31.31 GiB
2023-04-25 17:12:56,

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 8
Total threads: 16,Total memory: 250.47 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:39187,Workers: 8
Dashboard: http://127.0.0.1:8787/status,Total threads: 16
Started: Just now,Total memory: 250.47 GiB
Comm: tcp://127.0.0.1:46499,Total threads: 2
Dashboard: http://127.0.0.1:36249/status,Memory: 31.31 GiB
Nanny: tcp://127.0.0.1:37011,


In [3]:
tessellation = gpd.read_parquet("./out/singapore/tessellation_stats.pq")
buildings = gpd.read_parquet("./out/singapore/buildings.pq")

with open('./out/singapore/weights.pickle', 'rb') as f:
    queen_1 = pickle.load(f)
    queen_3 = pickle.load(f)
    dist200 = pickle.load(f)
    f.close()
    

In [4]:
percentiles = []
for column in tqdm(tessellation.columns.drop(["geometry", "uID"]), total = len(tessellation.columns.drop(["geometry", "uID"]))):
    perc = Percentiles(tessellation, column, queen_3, "uID", verbose=True, client=client).frame
    perc.columns = [f"{column}_" + str(x) for x in perc.columns]
    percentiles.append(perc)
    
percentiles.append(tessellation[["uID", "geometry"]])

  0%|          | 0/33 [00:00<?, ?it/s]

<string>:1: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/latest/migration_pygeos.html).
/home/virgilxw/miniconda3/lib/python3.10/site-packages/distributed/client.py:3095: UserWarning: Sending large graph of size 11.16 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(
<string>:1: UserWarning: Shapely 2.0 is installed, bu

KeyboardInterrupt: 

In [ ]:
percentiles_joined = pd.concat(percentiles, axis=1)
percentiles_joined.head()

,convexity_25,convexity_50,convexity_75,area_x_25,area_x_50,area_x_75,neighbours_25,neighbours_50,neighbours_75,neighbours400_25,...,global_clustering_400m_75,eigenvector_25,eigenvector_50,eigenvector_75,node_centrality_400m_25,node_centrality_400m_50,node_centrality_400m_75,betweeness_centrality_right_25,betweeness_centrality_right_50,betweeness_centrality_right_75
0,0.847505,0.942346,0.967193,19723.041136,37854.781814,114798.955260,18.0,22.0,25.0,2.0,...,0.187500,62345.0,63017.0,65197.5,2.0,16.0,20.0,0.000028,0.000056,0.000168
1,0.865144,0.923034,0.960621,13108.164245,31441.072713,64503.721598,26.0,34.0,41.0,2.0,...,0.079646,25573.0,25652.0,52676.0,12.0,24.0,31.0,0.000028,0.000196,0.000491
2,0.852574,0.925395,0.961069,8705.663720,28206.234361,59917.586944,30.0,36.0,42.0,2.0,...,0.188988,25574.0,25652.0,42869.0,13.5,26.0,30.0,0.000041,0.000196,0.000364
3,0.881916,0.928096,0.961455,7067.211140,21865.710257,53359.621793,31.0,38.0,46.0,3.0,...,0.079646,25574.0,25651.0,42869.0,12.5,26.0,29.0,0.000041,0.000252,0.000491
4,0.872610,0.927590,0.961008,6775.091831,22559.705001,56932.590132,32.5,38.0,46.0,2.5,...,0.195238,25577.0,25628.0,37914.0,12.0,17.0,28.0,0.000041,0.000196,0.000364


In [ ]:
percentiles_joined.to_parquet("./out/singapore/tessellation_percentiles.pq")

,convexity_25,convexity_50,convexity_75,area_x_25,area_x_50,area_x_75,neighbours_25,neighbours_50,neighbours_75,neighbours400_25,...,global_clustering_400m_75,eigenvector_25,eigenvector_50,eigenvector_75,node_centrality_400m_25,node_centrality_400m_50,node_centrality_400m_75,betweeness_centrality_right_25,betweeness_centrality_right_50,betweeness_centrality_right_75
0,-1.290818,0.232533,0.092995,2.443342,3.028067,4.028111,-1.083095,-1.043028,-1.103813,-1.060831,...,1.449506,2.399162,1.748317,1.088207,-1.564426,-1.230041,-1.283928,-0.188942,-0.266220,-0.393699
1,-0.791311,-0.591813,-0.295556,1.498087,2.441563,2.094302,-0.085882,0.211920,0.261412,-1.060831,...,0.107103,-0.018961,-0.503096,0.387329,-1.181615,-0.961646,-0.952306,-0.188942,-0.218522,-0.337602
2,-1.147271,-0.491037,-0.269117,0.868977,2.145752,1.917969,0.412724,0.421078,0.346738,-1.060831,...,1.468027,-0.018896,-0.503096,-0.161608,-1.124193,-0.894548,-0.982453,-0.180227,-0.218522,-0.359614
3,-0.316356,-0.375749,-0.246288,0.634845,1.565940,1.665821,0.537375,0.630236,0.688045,-1.035503,...,0.107103,-0.018896,-0.503156,-0.161608,-1.162474,-0.894548,-1.012601,-0.180227,-0.199447,-0.337602
4,-0.579881,-0.397346,-0.272726,0.593102,1.629403,1.803199,0.724353,0.630236,0.688045,-1.048167,...,1.545818,-0.018698,-0.504542,-0.438959,-1.181615,-1.196491,-1.042748,-0.180227,-0.218522,-0.359614
